In [12]:
# Importing all the required modules
import tensorrt as trt
import onnx
import torch
import numpy as np
import time
import torchvision
import torchvision.models as models
import pycuda.driver as cuda
import pycuda.autoinit

### Step 1: Convert the model to ONNX format

In [2]:
# Load the pretrained ResNet-50 model from torchvision library
model_resnet_50 = models.resnet50(pretrained=True)
model_resnet_50.eval()

/home/lkondap/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lkondap/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/lkondap/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:00<00:00, 148MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
# This block creates a dummy input and exports the model to ONNX
input_shape = (3,224,224)

dummy_input = torch.randn(1, *input_shape) # 1 here represents batch_size

# Emporting the model to ONNX
torch.onnx.export(model_resnet_50, dummy_input, "resnet_50.onnx", verbose=False)

# Notes: Setting the verbose to True is making the export run a lot slower

### Step 2: Optimize ONNX model with TensorRT

In [4]:
# Load the exported ONNX model
onnx_model = onnx.load("resnet_50.onnx")

# Create a TensorRT builder and network
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(1) # Notes 
config = builder.create_builder_config()
parser = trt.OnnxParser(network, TRT_LOGGER)
#builder.max_DLA_batch_size = 1
config.set_flag(trt.BuilderFlag.FP16)
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)

# Parse the ONNX model
success = parser.parse(onnx_model.SerializeToString())
if not success:
    for error in range(parser.num_errors):
        print(parser.get_error(error))
    exit()

# Build the TensorRT engine
engine = builder.build_serialized_network(network, config)

# Save the engine file
#with open("resnet_50.plan", "wb") as f:
#    f.write(engine)

#### Notes:
1. EXPLICIT_BATCH is deprecated as of TensorRT 10.0, otherwise an argument 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH) needs to be passed to create_network
2. 

In [5]:
# Create a runtime
trt_runtime = trt.Runtime(TRT_LOGGER)
# Deserialize engine
deserialized_eng = trt_runtime.deserialize_cuda_engine(engine)

In [6]:
def allocate_buffers(engine: trt.ICudaEngine):
    inputs = []
    outputs = []
    # Get IO names
    tensor_names = [engine.get_tensor_name(i) for i in range(engine.num_io_tensors)]
    for binding in tensor_names:
        dtype = np.dtype(trt.nptype(engine.get_tensor_dtype(binding)))
        # Host memory
        hostMemory = cuda.pagelocked_empty(trt.volume(engine.get_tensor_shape(binding)), dtype = dtype)
        # Device memory
        deviceMemory = cuda.mem_alloc(hostMemory.nbytes)
        if engine.get_tensor_mode(binding) == trt.TensorIOMode.INPUT:
            inputs.append(hostMemory)
            inputs.append(deviceMemory)
        else:
            outputs.append(hostMemory)
            outputs.append(deviceMemory)
    # Create a stream for inference
    stream = cuda.Stream()
    return inputs, outputs, stream

In [7]:
input_memory, output_memory, stream = allocate_buffers(deserialized_eng)

In [9]:
def do_inference(engine, pics, input_mem, output_mem, stream, batch_size, width):
  preprocessed = np.asarray(pics).ravel()
  np.copyto(image_mem[0], preprocessed)

  with engine.create_execution_context() as context:
      # Transfer input data to the GPU.
      cuda.memcpy_htod_async(input_mem[1], input_mem[0], stream)

      context.execute_v2(batch_size, bindings=[int(input_mem[1]), int(output_mem[1])])

      cuda.memcpy_dtoh_async(output_mem[0], output_mem[1], stream)
      # Synchronize the stream
      stream.synchronize()
      # Return the host output.
      out = output_mem[0].reshape((batch_size, 1, width))
      return out

In [20]:
image_path = "sample_images/n01440764_tench.JPEG"
image_string = torchvision.io.read_file(image_path)
image = torchvision.io.image.decode_jpeg(image_string, device='cuda')
image = torchvision.transforms.functional.convert_image_dtype(image, torch.float32)
image = torchvision.io.image.resize_image_with_crop_or_pad(image, 224, 224)

AttributeError: module 'torchvision.io.image' has no attribute 'resize_image_with_crop_or_pad'